<a href="https://colab.research.google.com/github/lishangli/DeepLearning/blob/master/Yolov1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
import torch.functional as F

architecture_config = [
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self,in_channels,out_channels,**kwargs):
    super(ConvBlock,self).__init__()
    self.conv = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,**kwargs)
    self.bn = nn.BatchNorm2d(out_channels)
    self.relu = nn.LeakyReLU(0.1)

  def forward(self,x):
    x = self.conv(x)
    x = self.bn(x)
    x = self.relu(x)
    return x 



In [ ]:
class Yolov1(nn.Module):
  def __init__(self,in_channel,out_channel,**kwargs):
    super(Yolov1,self).__init__()
	  self.in_channes = 3
    self.architecture = architecture_config
    self.body = self.create_body(self.architecture)

  def forward(self,x):
    x = self.body(x)
    x = torch.flattern(x,start_dim=1)
    return x

  def create_body(self,architecture):
    layers = []
    for arch in architecture:
      if type(arch) == tuple:
        layers += ConvBlock(in_channels =  self.in_channels,out_channels = arch[1],kernel_size = arch[0],stride = arch[2],padding = arch[3])
        self.in_channels = arch[1]
       
      elif type(arch) == str:
        layers += [nn.MaxPool2d(stride=2,kernel_size=2)]
        self.in_channels /= 2

      elif type(arch) == list:
        for ar in range(arch[-1]):

        	layers += ConvBlock(in_channels = self.in_channels,out_channels = arch[0][1],kernel_size = arch[0][0],stride = arch[0][2],padding = arch[0][3])
         	self.in_channels = arch[0][1]
           
          layers += ConvBlock(in_channels = self.in_channels,out_channels = arch[1][1],kerner_size = arch[1][0],stride = arch[1][2],padding = arch[1][3])
          self.in_channels = arch[1][1]
       
   return nn.Sequtional(*layers)

   







In [4]:
test = [[1,2],[2,3]]
for i in test:
  for j in i:
    print(j)

1
2
2
3
